In [ ]:
import csv
import numpy as np


with open(r'data/mnist_test.csv', 'r', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile)
    liver = np.array(list(spamreader))
    

data = (liver[1:, 1:]).astype(np.float64)
labels = (liver[1:, 0]).astype(np.int8)

print(data) 

In [ ]:
import csv
import numpy as np


with open(r'data/sonar.csv', 'r', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile)
    liver = np.array(list(spamreader))
    
raw_data= liver[:, :-1]
raw_labels = liver[:, -1]

# 创建标签映射
unique_labels = np.unique(raw_labels)
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
int_labels = np.array([label_to_int[label] for label in raw_labels], dtype=np.uint8)

print("raw_data:", raw_data)
# print("Raw labels:", raw_labels)
print("Integer labels:", int_labels)
# print("Label mapping:", label_to_int)

In [ ]:

import csv

# 打开原始CSV文件，读取数据并修改
with open('Relief_plain/data/colon.csv', 'r+', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    writer = csv.writer(csvfile, delimiter=',')
    
    # 读取每一行数据，删除第一个特征，并写回文件
    rows = [row for row in reader]
    csvfile.seek(0)  # 移动光标到文件开头
    
    for row in rows:
        if row:  # 确保行不为空
            new_row = row[1:]  # 删除第一个特征
            writer.writerow(new_row)
    
    csvfile.truncate()  # 截断文件至当前位置，确保删除可能残留的旧数据



In [ ]:
import crypten
import torch
import crypten.mpc as mpc
import crypten.communicator as comm 
from crypten.common.functions.logic import sign
import time

crypten.init()

@mpc.run_multiprocess(world_size=2)
def examine_arithmetic_shares():
    x_enc = crypten.cryptensor([10], ptype=crypten.mpc.arithmetic)
    
    rank = comm.get().get_rank()
    crypten.print(f"\nRank {rank}:\n {x_enc}\n", in_order=True)

    x_abs = sign(x_enc)
    print("x_abs", x_abs.get_plain_text())
        
x = examine_arithmetic_shares()



In [ ]:
import crypten
import torch
import crypten.mpc as mpc
import crypten.mpc.primitives.beaver as beaver
import crypten.communicator as comm 
import torch
from crypten.mpc import MPCTensor
from crypten.mpc.primitives import BinarySharedTensor
import time

crypten.init()

def generate_boolean_tensor(N):
    # 生成一个长度为N的随机布尔tensor
    random_tensor = torch.randint(0, 2, (N,))
    return random_tensor

def check_data_type(tensor):
    print(f"Type: {type(tensor)}")
    if isinstance(tensor, torch.Tensor):
        print(f"PyTorch Tensor dtype: {tensor.dtype}")
    elif isinstance(tensor, crypten.mpc.MPCTensor):
        print("This is a MPCTensor.")
    elif isinstance(tensor, crypten.mpc.primitives.binary.BinarySharedTensor):
        print("This is a BinarySharedTensor.")
    else:
        print("Unknown data type.")

N = 10
x = generate_boolean_tensor(N)
y = generate_boolean_tensor(N)
x_enc = crypten.cryptensor(x, ptype=crypten.mpc.binary)
y_enc = crypten.cryptensor(y, ptype=crypten.mpc.binary)
x_enc_share = x_enc.share
y_enc_share = y_enc.share
x_enc_binary = BinarySharedTensor(x_enc_share)
y_enc_binary = BinarySharedTensor(y_enc_share)
check_data_type(x_enc)
check_data_type(y_enc)
check_data_type(x_enc_binary)
check_data_type(y_enc_binary)

begin_time = time.time()
z_enc = x_enc_binary & y_enc_binary
end_time = time.time()

time = end_time - begin_time
    
# print("x_enc_type")
print("time:", time)
print("z_enc", z_enc)
print("z", z_enc.get_plain_text())

# examine_arithmetic_shares(x,y)



In [16]:
import crypten
import crypten.mpc
import torch
import crypten.mpc as mpc
import crypten.communicator as comm 
import crypten.common as common
import random
from crypten.mpc import MPCTensor
from crypten.mpc.primitives import BinarySharedTensor, ArithmeticSharedTensor
import crypten.mpc.primitives.beaver as beaver
from crypten.mpc.primitives.converters import convert
from crypten.mpc.ptype import ptype as Ptype

crypten.init()

def check_data_type(tensor):
    print(f"Type: {type(tensor)}")
    if isinstance(tensor, torch.Tensor):
        print(f"PyTorch Tensor dtype: {tensor.dtype}")
    elif isinstance(tensor, crypten.mpc.MPCTensor):
        print("This is a MPCTensor.")
    elif isinstance(tensor, crypten.mpc.primitives.binary.BinarySharedTensor):
        print("This is a BinarySharedTensor.")
    else:
        print("Unknown data type.")

def secret_sharing(x,a,b):
    x_enc = crypten.cryptensor(x, ptype = crypten.mpc.arithmetic)
    a_enc = crypten.cryptensor(a, ptype = crypten.mpc.arithmetic)
    b_enc = crypten.cryptensor(b, ptype = crypten.mpc.arithmetic)
    
    return x_enc, a_enc, b_enc


@mpc.run_multiprocess(world_size=2) 
def intvlTest(x, a, b):
    x_enc, a_enc, b_enc = secret_sharing(x, a, b)
    cmp_a_x = x_enc.ge(a_enc)
    cmp_x_b = b_enc.ge(x_enc)
    cmp_a_x_binary = cmp_a_x.to(crypten.mpc.binary)
    cmp_a_x_binary_share = cmp_a_x_binary.share
    cmp_a_x_binary_tensor = cmp_a_x_binary._tensor
    # cmp_a_x_binarytensor = BinarySharedTensor(cmp_a_x_binary_share)
    
    cmp_x_b_binary = cmp_x_b.to(crypten.mpc.binary)
    # cmp_x_b_binary_share = cmp_x_b_binary.share
    # cmp_x_b_binarytensor = BinarySharedTensor(cmp_x_b_binary_share)
    
    result_binary = cmp_a_x_binary._tensor & cmp_x_b_binary._tensor
    
    rank = comm.get().get_rank()
    # crypten.print(f"\nRank {rank}:\n before: {type(cmp_a_x_binary)}\n ", in_order=True)
    # crypten.print(f"\nRank {rank}:\n before: {type(cmp_a_x_binary_share)}\n ", in_order=True)
    # crypten.print(f"\nRank {rank}:\n before: {type(cmp_a_x_binary_tensor)}\n ", in_order=True)
    
    # crypten.print(f"\nRank {rank}:\n before: {cmp_a_x.get_plain_text()}\n after:  {cmp_a_x_binary.get_plain_text()}", in_order=True)
    # crypten.print(f"\nRank {rank}:\n before: {cmp_a_x_binarytensor.get_plain_text()}\n", in_order=True)
    crypten.print(f"\nRank {rank}:\n result_binary: {result_binary.get_plain_text()}\n", in_order=True)

@mpc.run_multiprocess(world_size=2) 
def test(a,b):
    a_share = BinarySharedTensor(a)
    b_share = BinarySharedTensor(b)
    c_share = a_share & b_share
    rank = comm.get().get_rank()
    crypten.print(f"\nRank {rank}:\n result: {type(b)}\n", in_order=True)
    # crypten.print(f"\nRank {rank}:\n result type: {type(c_share)}\n", in_order=True)
    # crypten.print(f"\nRank {rank}:\n result: {c_share.get_plain_text()}\n", in_order=True)

def main():
    # 区间范围
    a = [0,1,0]
    b = [1,0,1]
    
    #随机数范围
    x = random.randint(-10, 10) 
    # print("x", x)
    intvlTest(x, a, b)
    # test(a,b)
    
main()


Rank 0:
 result_binary: tensor([0, 0, 0])


Rank 1:
 result_binary: tensor([0, 0, 0])



In [52]:
import numpy as np

def generate_arrays(N, M):
    a = np.random.choice([-1, 1], size=N)
    b = np.random.randint(0, M+1, size=N)
    
    return a, b

def process_arrays(a, b):
    max_b = np.max(b)
    b_prime = max_b - b
    c = a * b_prime
    max_index = np.argmax(c)
    min_index = np.argmin(c)
    
    return c, max_index, min_index

def split_b_based_on_a(a, b):
    b1 = b[a == 1]
    b1_indices = np.where(a == 1)[0]
    b2 = b[a == -1]
    b2_indices = np.where(a == -1)[0]
    
    return b1, b1_indices, b2, b2_indices


N = 10
M = 50

a, b = generate_arrays(N, M)

c, max_index, min_index = process_arrays(a, b)

b1, b1_indices, b2, b2_indices = split_b_based_on_a(a, b)

# 打印结果
print("数组 a:", a)
print("数组 b:", b)
print("数组 b1 (a中对应值为1):", b1, "对应索引:", b1_indices, "最小值:", min(b1))
print("数组 b2 (a中对应值为-1):", b2, "对应索引:", b2_indices, "最小值:", min(b2))
print("数组 b':", max(b) - b)
print("数组 c:", c)
print("c中的最大值索引(near-hit):", max_index)
print("c中的最小值索引(near-miss):", min_index)



数组 a: [-1  1 -1 -1  1  1 -1 -1  1  1]
数组 b: [20 25  6 23 28 21 39 11  3 20]
数组 b1 (a中对应值为1): [25 28 21  3 20] 对应索引: [1 4 5 8 9] 最小值: 3
数组 b2 (a中对应值为-1): [20  6 23 39 11] 对应索引: [0 2 3 6 7] 最小值: 6
数组 b': [19 14 33 16 11 18  0 28 36 19]
数组 c: [-19  14 -33 -16  11  18   0 -28  36  19]
c中的最大值索引(near-hit): 8
c中的最小值索引(near-miss): 2


In [ ]:
import pandas as pd
import numpy as np

def generate_binary_dataset(m, n, filename):
    # 随机生成二元特征数据 (0 或 1)
    data = np.random.randint(2, size=(m, n))
    
    # 随机生成类别标签 ('N' 或 'Y')
    labels = np.random.choice(['N', 'Y'], size=(m,))
    
    # 将特征数据和标签结合起来
    df = pd.DataFrame(data, columns=[f'Feature_{i}' for i in range(n)])
    df['Label'] = labels
    
    # 将数据保存为 CSV 文件
    df.to_csv(filename, index=False)

# 示例使用
m = 100  # 样本数量
n = 3   # 特征数量
filename = 'binary_dataset.csv'

generate_binary_dataset(m, n, filename)
print(f"Dataset saved to {filename}")


In [ ]:
import crypten
import torch
import random

crypten.init()

feature = torch.tensor([[1, 0, 1, 0],
                        [0, 1, 0, 1],
                        [1, 1, 0, 0],
                        [0, 0, 1, 1]], dtype=torch.float32)

label = torch.tensor([0, 1, 1, 0], dtype=torch.float32)

y_share = crypten.cryptensor(feature, ptype=crypten.mpc.binary)
x_share = crypten.cryptensor(label, ptype=crypten.mpc.binary)

random_index = random.randint(0, feature.shape[0] - 1)

selected_sample_share = y_share[random_index]

selected_sample_plain = selected_sample_share.get_plain_text()

print("原始特征矩阵:")
print(feature)
print("随机索引:", random_index)
print("原始样本:", feature[random_index])
print("解密后的样本:", selected_sample_plain)

if torch.equal(feature[random_index], selected_sample_plain):
    print("解密后的样本与原始样本一致")
else:
    print("解密后的样本与原始样本不一致")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# 定义参数
t_eq = 1.0
t_diff = 2.0

# 定义 sigmoid 函数
def sigmoid(x, k, x0):
    return 1 / (1 + np.exp(-k * (x - x0)))

# 定义近似的分段函数
def approx_function(x, k1, k2):
    sigma1 = sigmoid(x, k1, t_eq)
    sigma2 = sigmoid(x, k2, t_diff)
    return sigma1 * (1 - sigma2) + sigma2

# 定义原始的分段函数
def piecewise_function(x):
    if x < t_eq:
        return 0
    elif x <= t_diff:
        return (x - t_eq) / (t_diff - t_eq)
    else:
        return 1

# 误差函数（均方误差）
def error_function(params, x):
    k1, k2 = params
    y_approx = approx_function(x, k1, k2)
    y_piecewise = np.array([piecewise_function(xi) for xi in x])
    return np.mean((y_approx - y_piecewise) ** 2)

# 初始猜测值
initial_guess = [10, 10]

# 优化过程
x = np.linspace(0, 3, 400)
result = minimize(error_function, initial_guess, args=(x,))
k1_opt, k2_opt = result.x

# 绘制结果
y_approx_opt = approx_function(x, k1_opt, k2_opt)
y_piecewise = np.array([piecewise_function(xi) for xi in x])

plt.figure(figsize=(10, 6))
plt.plot(x, y_approx_opt, label=f'Optimized Approximate Function (k1={k1_opt:.2f}, k2={k2_opt:.2f})', linestyle='--')
plt.plot(x, y_piecewise, label='Original Piecewise Function')
plt.title('Optimization of Sigmoid Approximation to Piecewise Function')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def L_Sigmoid(x, min_value, max_value):
    if x < min_value:
        return 0
    elif x > max_value:
        return 1
    else:
        return x

# 定义x的范围
x_values = np.linspace(-10, 10, 400)
y_values = np.array([L_Sigmoid(x, 2, 8) for x in x_values])

# 创建一个掩码来去除分段中的垂直线
mask = (x_values >= 2) & (x_values <= 8)
x_values_segment = x_values[mask]
y_values_segment = y_values[mask]

plt.plot(x_values[x_values < 2], y_values[x_values < 2], 'b')
plt.plot(x_values_segment, y_values_segment, 'b')
plt.plot(x_values[x_values > 8], y_values[x_values > 8], 'b')

plt.xlabel('x')
plt.ylabel('L_Sigmoid(x)')
plt.title('Piecewise Linear Sigmoid Function')
plt.grid(True)
plt.show()


In [21]:
import os
import numpy as np
import csv
 

DATASET_DIR = os.path.join(os.path.dirname(__file__), 'dataset/')

class DataSetParam:
    def __init__(self, dataset_name):
        self.dataset_name = dataset_name

class DataSet:
    sonar = DataSetParam('sonar.csv')
    binary_dataset = DataSetParam("binary_dataset.csv")
    
def load_dataset(dataset):
    dataset_path = os.path.join(DATASET_DIR, dataset)
    try:
        with open(dataset_path, 'r', encoding='utf-8') as csvfile:
            spamreader = csv.reader(csvfile)
            data = np.array(list(spamreader))
    except FileNotFoundError:
        print(f"Error: The file {dataset_path} was not found.")
        return None, None
    except Exception as e:
        print(f"Error: An error occurred while reading the file: {e}")
        return None, None

    if data.size == 0:
        print("Error: The dataset is empty.")
        return None, None

    feature = data[:, :-1].astype(np.float64)
    labels = data[:, -1]
    
    # 创建标签映射
    unique_labels = np.unique(labels)
    label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
    int_labels = np.array([label_to_int[label] for label in labels], dtype=np.uint8)
    
    # 将标签转换为独热矩阵
    num_classes =  len(unique_labels)
    one_hot_labels = np.zeros((int_labels.size, num_classes), dtype=np.uint8)
    one_hot_labels[np.arange(int_labels.size), int_labels] = 1

    return feature, one_hot_labels

data_train = DataSet.binary_dataset 
feature, label = load_dataset(data_train.dataset_name)

NameError: name '__file__' is not defined